In [2]:
import pandas as pd 
import json
import numpy as np 
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [3]:
import os 


In [4]:
%pwd

'c:\\Users\\abhis\\OneDrive\\Desktop\\Oil_Retail\\research'

In [7]:
df = pd.read_csv(r"C:\Users\abhis\OneDrive\Desktop\Oil_Retail\data\oil_retail_history.csv")
df.head()

,date,price,cost,comp1_price,comp2_price,comp3_price,volume
0,2023-01-01,97.66,86.46,98.39,99.10,97.37,13513
1,2023-01-02,96.46,87.25,95.37,96.25,97.38,13605
2,2023-01-03,94.67,86.40,94.41,94.69,94.50,16540
3,2023-01-04,97.32,87.51,99.12,96.90,97.54,14927
4,2023-01-05,94.08,86.38,93.31,94.61,93.85,13332


In [6]:
df.shape

(730, 7)

In [8]:
df.dtypes

date            object
price          float64
cost           float64
comp1_price    float64
comp2_price    float64
comp3_price    float64
volume           int64
dtype: object

In [9]:
df.isnull().sum()

date           0
price          0
cost           0
comp1_price    0
comp2_price    0
comp3_price    0
volume         0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         730 non-null    object 
 1   price        730 non-null    float64
 2   cost         730 non-null    float64
 3   comp1_price  730 non-null    float64
 4   comp2_price  730 non-null    float64
 5   comp3_price  730 non-null    float64
 6   volume       730 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 40.0+ KB


In [12]:
df.duplicated().sum()

0

In [13]:
df.describe()

,price,cost,comp1_price,comp2_price,comp3_price,volume
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,96.505301,86.498178,96.551658,96.530068,96.457260,13960.983562
std,2.747598,1.575689,2.779654,2.796763,2.782295,905.912360
min,88.080000,83.340000,88.320000,87.070000,86.080000,11397.000000
25%,94.670000,85.302500,94.555000,94.827500,94.712500,13326.250000
50%,96.525000,86.255000,96.590000,96.430000,96.515000,13882.000000
75%,98.420000,87.610000,98.467500,98.447500,98.097500,14538.750000
max,104.890000,91.860000,104.200000,105.000000,104.360000,16890.000000
